In [261]:
import pandas
import datetime

def get_cleaned_date(date):
    """
    Return datetime object from a string
    """
    date = str(date).strip()

    if '&' in date:
        date = date.split('&')[0].strip()

    # Since there are a lot of formats in the data, need to handle all the possible options
    date_formats = [
        '%d.%m.%Y', '%d.%m.%y', '%d.%m.%y', '%d-%m-%Y', '%d/%m/%y', '%d/%m/%Y', '%d %b %y', '%d-%b -%y',
        '%d – %b-%y', '%d -%b -%y'
    ]

    for date_format in date_formats:
        try:
            return datetime.datetime.strptime(date, date_format)
        except ValueError:
            pass

def get_cleaned_options(string, options, result, target):
    if (any(st in string.lower() for st in options)):
        target.append(result)
    return target

spamreader = rawData = pandas.read_csv('Interview.csv')

#Remove columns that provide no useful data.
cleanData = rawData.drop(['Name(Cand ID)', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27'], axis = 1)

#Remove rows that provide no useful data
cleanData = cleanData.drop(1233)

#Change column names for easier reference
cleanData.columns = [
    'date',
    'client',
    'industry',
    'loc',
    'pos',
    'skillset',
    'interview_type',
    'gender',
    'current_loc',
    'job_loc',
    'venue_loc',
    'native_loc',
    'has_permission',
    'unscheduled',
    'can_call',
    'can_hava_alt_number',
    'has_cv_print',
    'venue',
    'call_letter_shared',
    'expected_attendance',
    'observed_attendance',
    'marital_status']

#Fix date formats
cleanData['date'] = cleanData['date'].map(get_cleaned_date)

#Make everything lower case and strip whitespace for more uniform data
cleanData = pandas.concat([cleanData[c].astype(str).str.lower() for c in cleanData.columns], axis = 1)
cleanData = pandas.concat([cleanData[c].astype(str).str.strip() for c in cleanData.columns], axis = 1)

#Merge standard chartered bank and standard chartered bank chennai. job location is specified in another column.
cleanData['client'].replace('standard chartered bank chennai', 'standard chartered bank', inplace = True)

#aon hewitt, hewitt and aon hewitt gurgaon seem to be the same company, we merge those too
cleanData['client'].replace(['hewitt', 'aon hewitt gurgaon'], 'aon hewitt', inplace = True)

#Merge similar industry items
cleanData['industry'].replace(['it products and services', 'it services'], 'it', inplace = True)

#Fix interview types
cleanData['interview_type'].replace(['scheduled walk in', 'sceduled walkin'], 'scheduled walkin', inplace=True)

#Fix location fields
cleanData['loc'].replace(['- cochin-', 'gurgaonr'], ['cochin', 'gurgaon'], inplace = True)
cleanData['current_loc'].replace('- cochin-', 'cochin', inplace=True)
cleanData['job_loc'].replace('- cochin-', 'cochin', inplace=True)
cleanData['venue_loc'].replace('- cochin-', 'cochin', inplace=True)
cleanData['native_loc'].replace('- cochin-', 'cochin', inplace=True)
cleanData['native_loc'].replace('delhi /ncr', 'delhi', inplace=True)

#Fix yes/no/na fields
no = ['not yet', 'no dont', 'no- will take it soon', 'no i have only thi number', 'no- i need to check']
na = ['nan', 'not sure', 'cant say', 'yet to confirm', 'need to check', 'yet to check', 'havent checked', 'uncertain']
cleanData['has_permission'].replace(no, 'no', inplace=True)
cleanData['has_permission'].replace(na, 'na', inplace=True)
cleanData['unscheduled'].replace(na, 'na', inplace=True)
cleanData['can_call'].replace(na, 'na', inplace=True)
cleanData['can_call'].replace(no, 'no', inplace=True)
cleanData['can_hava_alt_number'].replace(na, 'na', inplace=True)
cleanData['can_hava_alt_number'].replace(no, 'no', inplace=True)
cleanData['has_cv_print'].replace(na, 'na', inplace=True)
cleanData['has_cv_print'].replace(no, 'no', inplace=True)
cleanData['venue'].replace(na, 'na', inplace=True)
cleanData['venue'].replace(no, 'no', inplace=True)
cleanData['call_letter_shared'].replace(na, 'na', inplace=True)
cleanData['call_letter_shared'].replace(no, 'no', inplace=True)
cleanData['expected_attendance'].replace(na, 'na', inplace=True)
cleanData['expected_attendance'].replace(no, 'no', inplace=True)

#We will assume that the time means that the candidate is expected at that time
cleanData['expected_attendance'].replace('\d\d[.:]\d\d [ap]m', 'yes', regex=True, inplace=True)

cleanData['skillset'].value_counts()
#TODO: deal with the skillset field separating each value into keywords and then binarize.
#We don't use onehot because we don't know which algorithm will be used yet

java/j2ee/struts/hibernate                220
accounting operations                      86
fresher                                    86
aml/kyc/cdd                                84
cdd kyc                                    52
java j2ee                                  49
routine                                    47
oracle                                     43
java/spring/hibernate/jsf                  42
java                                       31
sas                                        27
oracle plsql                               25
java developer                             25
lending and liabilities                    25
banking operations                         24
core java                                  17
sccm                                       15
t-24 developer                             15
senior software engineer-mednet            15
als testing                                15
analytical r & d                           13
cots developer                    

In [ ]:

col = []
data = []
for row in spamreader:

    date = get_cleaned_date(row[0])
    if date is None:
        continue
    row[0] = date

    if (row[2] in ['IT Services', 'IT Products and Services']):
        row[2] = 'IT'

    row[3] = get_cleaned_location(row[3])

    tags = []

    if (timeFormat.match(row[5]) is not None):
        continue
    if (timeFormat.match(row[20]) is not None):
        continue

    get_cleaned_options(row[5], ['sr', 'senior'], 'Senior', tags)
    get_cleaned_options(row[5], ['als testing'], 'ALS Testing', tags)
    get_cleaned_options(row[5], ['analytical r & d', 'analytical r&d'], 'Analytical R&D', tags)
    get_cleaned_options(row[5], ['accounting operations'], 'Accounting Operations', tags)
    get_cleaned_options(row[5], ['banking operations'], 'Banking Operations', tags)
    get_cleaned_options(row[5], ['cdd'], 'CDD', tags)
    get_cleaned_options(row[5], ['kyc'], 'KYC', tags)
    get_cleaned_options(row[5], ['aml'], 'AML', tags)
    get_cleaned_options(row[5], ['biosimiliar', 'biosimiliars', 'biosimilars', 'biosimillar'], 'Biosimilar', tags)
    get_cleaned_options(row[5], ['cots'], 'COTS', tags)
    get_cleaned_options(row[5], ['dot net'], '.Net', tags)
    get_cleaned_options(row[5], ['java'], 'Java', tags)
    get_cleaned_options(row[5], ['sql'], 'SQL', tags)
    get_cleaned_options(row[5], ['j2ee'], 'J2EE', tags)
    get_cleaned_options(row[5], ['sapbo'], 'SAPBO', tags)
    get_cleaned_options(row[5], ['informatica'], 'Informatica', tags)
    get_cleaned_options(row[5], ['struts'], 'Struts', tags)
    get_cleaned_options(row[5], ['hibernate'], 'Hibernate', tags)
    get_cleaned_options(row[5], ['jsf'], 'JSF', tags)
    get_cleaned_options(row[5], ['sas'], 'SAS', tags)
    get_cleaned_options(row[5], ['xml'], 'XML', tags)
    get_cleaned_options(row[5], ['spring'], 'Spring', tags)
    get_cleaned_options(row[5], ['automation testing'], 'Automation Testing', tags)
    get_cleaned_options(row[5], ['tech lead', 'technical lead', 'tl'], 'Technical Lead', tags)
    get_cleaned_options(row[5], ['l & l', 'lending & liability', 'lending and liabilities', 'lending&liablities'], 'Lending and Liability', tags)
    get_cleaned_options(row[5], ['etl'], 'ETL', tags)
    get_cleaned_options(row[5], ['fresher'], 'Fresher', tags)
    get_cleaned_options(row[5], ['global labelling'], 'Global Labeling', tags)
    get_cleaned_options(row[5], ['hadoop'], 'Hadoop', tags)
    get_cleaned_options(row[5], ['emea'], 'EMEA', tags)
    get_cleaned_options(row[5], ['licensing'], 'Licensing', tags)
    get_cleaned_options(row[5], ['manager'], 'Manager', tags)
    get_cleaned_options(row[5], ['lcm'], 'Life Cycle Management', tags)
    get_cleaned_options(row[5], ['oracle'], 'Oracle', tags)
    get_cleaned_options(row[5], ['production'], 'Production', tags)
    get_cleaned_options(row[5], ['support'], 'Support', tags)
    get_cleaned_options(row[5], ['sccm'], 'SCCM', tags)
    get_cleaned_options(row[5], ['sharepoint'], 'Sharepoint', tags)
    get_cleaned_options(row[5], ['network'], 'Network', tags)
    get_cleaned_options(row[5], ['ms exchange'], 'MS Exchange', tags)
    get_cleaned_options(row[5], ['product control'], 'Product Control', tags)
    get_cleaned_options(row[5], ['publishing'], 'Publishing', tags)
    get_cleaned_options(row[5], ['label'], 'Label', tags)
    get_cleaned_options(row[5], ['regulatory'], 'Regulatory', tags)
    get_cleaned_options(row[5], ['routine'], 'Routine', tags)
    get_cleaned_options(row[5], ['analyst'], 'Analist', tags)
    get_cleaned_options(row[5], ['software engineer'], 'Software Engineer', tags)
    get_cleaned_options(row[5], ['mednet'], 'Mednet', tags)
    get_cleaned_options(row[5], ['submission management'], 'Submission Management', tags)
    get_cleaned_options(row[5], ['t-24 developer'], 'T-24 Developer', tags)
    get_cleaned_options(row[5], ['generic drugs'], 'Generic Drugs', tags)
    get_cleaned_options(row[5], ['testing'], 'Testing', tags)
    if (all(st in row[5] for st in ['BaseSAS', 'Program'])):
        tags.append('BaseSAS Program')
    if (all(st in row[5] for st in ['BaseSAS', 'Reporting'])):
        tags.append('BaseSAS Reporting')
